In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import grangercausalitytests
from nodiensenv.constants import DATA_DIR, FIGURE_DIR
from nodiensenv.analyser import TrendPriceAnalyser

years = [2021, 2022, 2023, 2024]

for year in years:
    trend = "dogecoin"
    coin_name = "DOGE"
    trend_h1 = pd.read_csv(
        DATA_DIR / f"trend_{coin_name}_{year}_H1.csv", parse_dates=["date"]
    )
    trend_h2 = pd.read_csv(
        DATA_DIR / f"trend_{coin_name}_{year}_H2.csv", parse_dates=["date"]
    )
    price = pd.read_csv(DATA_DIR / f"{coin_name}_price_mcap.csv", parse_dates=["date"])

In [ ]:
trend_h1.head()

,date,dogecoin
0,2024-01-01,13
1,2024-01-02,16
2,2024-01-03,15
3,2024-01-04,14
4,2024-01-05,12


In [ ]:
price.head()

,date,open,high,low,close,volumefrom,volumeto,supply,market_cap
0,2021-01-01,0.004619,0.005741,0.004606,0.005699,3.179644e+08,1639311.25,1.277622e+11,7.281167e+08
1,2021-01-02,0.005699,0.014080,0.005597,0.010580,3.708311e+09,34174268.90,1.277761e+11,1.351872e+09
2,2021-01-03,0.010580,0.013860,0.009254,0.009826,2.050835e+09,23319659.14,1.277902e+11,1.255667e+09
3,2021-01-04,0.009826,0.011510,0.007388,0.009776,1.490013e+09,14197090.74,1.278040e+11,1.249412e+09
4,2021-01-05,0.009776,0.010300,0.008825,0.009994,3.949869e+08,3784489.68,1.278178e+11,1.277411e+09


In [ ]:
price_year = price[price["date"].dt.year == year].sort_values("date")
price_h1 = price_year[(price_year["date"] <= f"{year}-06-30")].copy()
price_h2 = price_year[(price_year["date"] >= f"{year}-07-01")].copy()

for df in (price_h1, price_h2):
    df["log_return"] = np.log(df["close"] / df["close"].shift(1)) * 100
    df["abs_log_return"] = abs(df["log_return"])

for df in (trend_h1, trend_h2):
    df["trend_log_return"] = np.log(df[f"{trend}"] / df[f"{trend}"].shift(1)) * 100
    df["abs_trend_log_return"] = abs(df["trend_log_return"])

In [ ]:
price_h1.head()

,date,open,high,low,close,volumefrom,volumeto,supply,market_cap,log_return,abs_log_return
1095,2024-01-01,0.08950,0.09208,0.08850,0.09200,1.583649e+08,14363090.41,1.424803e+11,1.310819e+10,NaN,NaN
1096,2024-01-02,0.09200,0.09441,0.09042,0.09110,3.562606e+08,33026702.48,1.424936e+11,1.298117e+10,-0.983077,0.983077
1097,2024-01-03,0.09110,0.09213,0.07702,0.08203,7.131253e+08,59961786.12,1.425072e+11,1.168987e+10,-10.487277,10.487277
1098,2024-01-04,0.08203,0.08475,0.08112,0.08408,3.364691e+08,27993882.62,1.425342e+11,1.198427e+10,2.468369,2.468369
1099,2024-01-05,0.08408,0.08490,0.08041,0.08272,4.966445e+08,40946885.60,1.425342e+11,1.179043e+10,-1.630732,1.630732


In [ ]:
trend_h1.head()

,date,dogecoin,trend_log_return,abs_trend_log_return
0,2024-01-01,13,NaN,NaN
1,2024-01-02,16,20.763936,20.763936
2,2024-01-03,15,-6.453852,6.453852
3,2024-01-04,14,-6.899287,6.899287
4,2024-01-05,12,-15.415068,15.415068


In [ ]:
df_h1 = pd.merge(
    trend_h1,
    price_h1[["date", "close", "log_return", "abs_log_return"]],
    on="date",
    how="inner",
)
df_h2 = pd.merge(
    trend_h2,
    price_h2[["date", "close", "log_return", "abs_log_return"]],
    on="date",
    how="inner",
)

In [ ]:
df_h1.head()

,date,dogecoin,trend_log_return,abs_trend_log_return,close,log_return,abs_log_return
0,2024-01-01,13,NaN,NaN,0.09200,NaN,NaN
1,2024-01-02,16,20.763936,20.763936,0.09110,-0.983077,0.983077
2,2024-01-03,15,-6.453852,6.453852,0.08203,-10.487277,10.487277
3,2024-01-04,14,-6.899287,6.899287,0.08408,2.468369,2.468369
4,2024-01-05,12,-15.415068,15.415068,0.08272,-1.630732,1.630732


In [ ]:
series_price_h1 = df_h1[["log_return"]].dropna()
series_trend_h1 = df_h1[["trend_log_return"]].dropna()

In [ ]:
series_price_h1

,log_return
1,-0.983077
2,-10.487277
3,2.468369
4,-1.630732
5,-2.559062
...,...
177,-2.733289
178,3.365702
179,-3.284236
180,-0.899802


In [ ]:
series_price_h1 = df_h1[["log_return"]].dropna()
series_trend_h1 = df_h1[["trend_log_return"]].dropna()
df_gc = pd.concat([series_price_h1, series_trend_h1], axis=1).dropna()
df_gc.columns = ["price_ret", "trend_ret"]

"""
Runs Granger tests up to `max_lag` for both directions:
1) Does trend --> price?
2) Does price --> trend?
Prints a summary for each.
"""

def run_granger(df, max_lag):
    print(f"\n=== Testing Granger causality up to lag={max_lag} ===\n")

    # 3a) trend --> price
    print(">> Does TREND Granger-cause PRICE?")
    test_result_1 = grangercausalitytests(df[["price_ret","trend_ret"]], 
                                        maxlag=max_lag, verbose=True)
    # 3b) price --> trend
    print("\n>> Does PRICE Granger-cause TREND?")
    test_result_2 = grangercausalitytests(df[["trend_ret","price_ret"]], 
                                        maxlag=max_lag, verbose=True)

    return test_result_1, test_result_2

res_trend_to_price, res_price_to_trend = run_granger(df_gc, max_lag=3)


=== Testing Granger causality up to lag=3 ===

>> Does TREND Granger-cause PRICE?

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0676  , p=0.7952  , df_denom=177, df_num=1
ssr based chi2 test:   chi2=0.0687  , p=0.7932  , df=1
likelihood ratio test: chi2=0.0687  , p=0.7932  , df=1
parameter F test:         F=0.0676  , p=0.7952  , df_denom=177, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.3201  , p=0.7265  , df_denom=174, df_num=2
ssr based chi2 test:   chi2=0.6586  , p=0.7194  , df=2
likelihood ratio test: chi2=0.6574  , p=0.7198  , df=2
parameter F test:         F=0.3201  , p=0.7265  , df_denom=174, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=1.6487  , p=0.1800  , df_denom=171, df_num=3
ssr based chi2 test:   chi2=5.1486  , p=0.1612  , df=3
likelihood ratio test: chi2=5.0756  , p=0.1663  , df=3
parameter F test:         F=1.6487  , p=0.1800  , df_denom=171, df_num=3

>> Does PR

/Users/dsf-air-17/Dev/nodiens_index/venv/lib/python3.13/site-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/Users/dsf-air-17/Dev/nodiens_index/venv/lib/python3.13/site-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(


In [ ]:
year

2024

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import grangercausalitytests
from nodiensenv.constants import DATA_DIR, FIGURE_DIR
from nodiensenv.analyser import TrendPriceAnalyser

years = [2021, 2022, 2023, 2024]

for year in years:
    trend = "bitcoin"
    coin_name = "BTC"
    trend_h1 = pd.read_csv(
        DATA_DIR / f"trend_{coin_name}_{year}_H1.csv", parse_dates=["date"]
    )
    trend_h2 = pd.read_csv(
        DATA_DIR / f"trend_{coin_name}_{year}_H2.csv", parse_dates=["date"]
    )
    price = pd.read_csv(DATA_DIR / f"{coin_name}_price_mcap.csv", parse_dates=["date"])

    # Filter price to {year}, split H1 & H2, and compute daily log-returns
    price_year = price[price["date"].dt.year == year].sort_values("date")
    price_h1 = price_year[(price_year["date"] <= f"{year}-06-30")].copy()
    price_h2 = price_year[(price_year["date"] >= f"{year}-07-01")].copy()

    for df in (price_h1, price_h2):
        df["log_return"] = np.log(df["close"] / df["close"].shift(1)) * 100
        df["abs_log_return"] = abs(df["log_return"])

    for df in (trend_h1, trend_h2):
        df["trend_log_return"] = np.log(df[f"{trend}"] / df[f"{trend}"].shift(1)) * 100
        df["abs_trend_log_return"] = abs(df["trend_log_return"])

    # Merge each half’s trends with its prices
    df_h1 = pd.merge(
        trend_h1,
        price_h1[["date", "close", "log_return", "abs_log_return"]],
        on="date",
        how="inner",
    )
    df_h2 = pd.merge(
        trend_h2,
        price_h2[["date", "close", "log_return", "abs_log_return"]],
        on="date",
        how="inner",
    )

    series_price_h1 = df_h1[["log_return"]].dropna()
    series_trend_h1 = df_h1[["trend_log_return"]].dropna()
    df_gc = pd.concat([series_price_h1, series_trend_h1], axis=1).dropna()
    df_gc.columns = ["price_ret", "trend_ret"]

    def run_granger(df, max_lag):
        print(f"\n=== Testing Granger causality up to lag={max_lag} in {year} ===\n")

        # 3a) trend --> price
        print(">> Does TREND Granger-cause PRICE?")
        test_result_1 = grangercausalitytests(
            df[["price_ret", "trend_ret"]], maxlag=max_lag, verbose=True
        )
        # 3b) price --> trend
        print("\n>> Does PRICE Granger-cause TREND?")
        test_result_2 = grangercausalitytests(
            df[["trend_ret", "price_ret"]], maxlag=max_lag, verbose=True
        )

        return test_result_1, test_result_2

    res_trend_to_price, res_price_to_trend = run_granger(df_gc, max_lag=3)

    # analyser = TrendPriceAnalyser(trend, df_h1, df_h2)

    # for smooth_window in [3]:
    #         analyser.plot_smoothed(
    #             "H1",
    #             "close",
    #             f"{trend}",
    #             smooth_window,
    #             "Close Price",
    #             f"Google Trend ({trend})",
    #             f"H1 {year}: {coin_name} Close vs Trend (rolling window={smooth_window})",
    #         )
    #         analyser.plot_smoothed(
    #             "H2",
    #             "close",
    #             f"{trend}",
    #             smooth_window,
    #             "Close Price",
    #             f"Google Trend ({trend})",
    #             f"H2 {year}: {coin_name} Close vs Trend (rolling window={smooth_window})",
    #         )
    #     analyser.plot_smoothed(
    #         "H1",
    #         "log_return",
    #         f"{trend}",
    #         smooth_window,
    #         "Log Return",
    #         f"Google Trend ({trend})",
    #         f"H1 {year}: {coin_name} Log Return vs Trend (rolling window={smooth_window})",
    #     )
    #     analyser.plot_smoothed(
    #         "H2",
    #         "log_return",
    #         f"{trend}",
    #         smooth_window,
    #         "Log Return",
    #         f"Google Trend ({trend})",
    #         f"H2 {year}: {coin_name} Log Return vs Trend (rolling window={smooth_window})",
    #     )
    #     analyser.plot_smoothed(
    #         "H1",
    #         "abs_log_return",
    #         f"{trend}",
    #         smooth_window,
    #         "Log Return",
    #         f"Google Trend ({trend})",
    #         f"H1 {year}: {coin_name} abs Log Return vs Trend (rolling window={smooth_window})",
    #     )
    #     analyser.plot_smoothed(
    #         "H2",
    #         "abs_log_return",
    #         f"{trend}",
    #         smooth_window,
    #         "Log Return",
    #         f"Google Trend ({trend})",
    #         f"H2 {year}: {coin_name} abs Log Return vs Trend (rolling window={smooth_window})",
    #     )
    #     analyser.plot_smoothed(
    #         "H1",
    #         "log_return",
    #         "trend_log_return",
    #         smooth_window,
    #         "Price Log Return",
    #         "Trend Log Return",
    #         f"H1 {year}: {coin_name} Price Log Return vs Trend Log Return (rolling window={smooth_window})",
    #     )
    #     analyser.plot_smoothed(
    #         "H2",
    #         "log_return",
    #         "trend_log_return",
    #         smooth_window,
    #         "Price Log Return",
    #         "Trend Log Return",
    #         f"H2 {year}: {coin_name} Price Log Return vs Trend Log Return (rolling window={smooth_window})",
    #     )
    # analyser.plot_smoothed(
    #     "H1",
    #     "abs_log_return",
    #     "abs_trend_log_return",
    #     smooth_window,
    #     "Price Abs Log Return",
    #     f"Trend Abs Log Return({trend})",
    #     f"H1 {year}: {coin_name} Price Abs Log Return vs Trend Abs Log Return (rolling window={smooth_window})",
    # )
    # analyser.plot_smoothed(
    #     "H2",
    #     "abs_log_return",
    #     "abs_trend_log_return",
    #     smooth_window,
    #     "Price Abs Log Return",
    #     f"Trend Abs Log Return ({trend})",
    #     f"H2 {year}: {coin_name} Price Abs Log Return vs Trend Abs Log Return (rolling window={smooth_window})",
    # )


=== Testing Granger causality up to lag=3 in 2021 ===

>> Does TREND Granger-cause PRICE?

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=2.1175  , p=0.1474  , df_denom=176, df_num=1
ssr based chi2 test:   chi2=2.1536  , p=0.1422  , df=1
likelihood ratio test: chi2=2.1407  , p=0.1434  , df=1
parameter F test:         F=2.1175  , p=0.1474  , df_denom=176, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=1.4838  , p=0.2296  , df_denom=173, df_num=2
ssr based chi2 test:   chi2=3.0533  , p=0.2173  , df=2
likelihood ratio test: chi2=3.0274  , p=0.2201  , df=2
parameter F test:         F=1.4838  , p=0.2296  , df_denom=173, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=1.3808  , p=0.2504  , df_denom=170, df_num=3
ssr based chi2 test:   chi2=4.3129  , p=0.2296  , df=3
likelihood ratio test: chi2=4.2612  , p=0.2346  , df=3
parameter F test:         F=1.3808  , p=0.2504  , df_denom=170, df_num=3

>>

/Users/dsf-air-17/Dev/nodiens_index/venv/lib/python3.13/site-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/Users/dsf-air-17/Dev/nodiens_index/venv/lib/python3.13/site-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/Users/dsf-air-17/Dev/nodiens_index/venv/lib/python3.13/site-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/Users/dsf-air-17/Dev/nodiens_index/venv/lib/python3.13/site-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/Users/dsf-air-17/Dev/nodiens_index/venv/lib/python3.13/site-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(


In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import grangercausalitytests
from nodiensenv.constants import DATA_DIR, FIGURE_DIR
from nodiensenv.analyser import TrendPriceAnalyser

years = [2021, 2022, 2023, 2024]

for year in years:
    trend = "bitcoin"
    coin_name = "BTC"
    trend_h1 = pd.read_csv(
        DATA_DIR / f"trend_{coin_name}_{year}_H1.csv", parse_dates=["date"]
    )
    trend_h2 = pd.read_csv(
        DATA_DIR / f"trend_{coin_name}_{year}_H2.csv", parse_dates=["date"]
    )
    price = pd.read_csv(DATA_DIR / f"{coin_name}_price_mcap.csv", parse_dates=["date"])

    # Filter price to {year}, split H1 & H2, and compute daily log-returns
    price_year = price[price["date"].dt.year == year].sort_values("date")
    price_h1 = price_year[(price_year["date"] <= f"{year}-06-30")].copy()
    price_h2 = price_year[(price_year["date"] >= f"{year}-07-01")].copy()

    for df in (price_h1, price_h2):
        df["log_return"] = np.log(df["close"] / df["close"].shift(1)) * 100
        df["abs_log_return"] = abs(df["log_return"])

    for df in (trend_h1, trend_h2):
        df["trend_log_return"] = np.log(df[f"{trend}"] / df[f"{trend}"].shift(1)) * 100
        df["abs_trend_log_return"] = abs(df["trend_log_return"])

    # Merge each half’s trends with its prices
    df_h1 = pd.merge(
        trend_h1,
        price_h1[["date", "close", "log_return", "abs_log_return"]],
        on="date",
        how="inner",
    )
    df_h2 = pd.merge(
        trend_h2,
        price_h2[["date", "close", "log_return", "abs_log_return"]],
        on="date",
        how="inner",
    )

    # series_price_h1 = df_h1[["log_return"]].dropna()
    # series_trend_h1 = df_h1[["trend_log_return"]].dropna()
    series_price_h1 = df_h1[["close"]].dropna()
    series_trend_h1 = df_h1[[f"{trend}"]].dropna()
    df_gc = pd.concat([series_price_h1, series_trend_h1], axis=1).dropna()
    df_gc.columns = ["price_ret", "trend_ret"]

    # Runs Granger tests up to `max_lag` for both directions:
    #     1) Does trend --> price?
    #     2) Does price --> trend?

    def run_granger(df, max_lag):
        print(f"\n=== Testing Granger causality up to lag={max_lag} in {year} ===\n")

        # 3a) trend --> price
        print(">> Does TREND Granger-cause PRICE?")
        test_result_1 = grangercausalitytests(
            df[["price_ret", "trend_ret"]], maxlag=max_lag, verbose=True
        )
        # 3b) price --> trend
        print("\n>> Does PRICE Granger-cause TREND?")
        test_result_2 = grangercausalitytests(
            df[["trend_ret", "price_ret"]], maxlag=max_lag, verbose=True
        )

        return test_result_1, test_result_2

    res_trend_to_price, res_price_to_trend = run_granger(df_gc, max_lag=3)

    # analyser = TrendPriceAnalyser(trend, df_h1, df_h2)

    # for smooth_window in [3]:
    #         analyser.plot_smoothed(
    #             "H1",
    #             "close",
    #             f"{trend}",
    #             smooth_window,
    #             "Close Price",
    #             f"Google Trend ({trend})",
    #             f"H1 {year}: {coin_name} Close vs Trend (rolling window={smooth_window})",
    #         )
    #         analyser.plot_smoothed(
    #             "H2",
    #             "close",
    #             f"{trend}",
    #             smooth_window,
    #             "Close Price",
    #             f"Google Trend ({trend})",
    #             f"H2 {year}: {coin_name} Close vs Trend (rolling window={smooth_window})",
    #         )
    #     analyser.plot_smoothed(
    #         "H1",
    #         "log_return",
    #         f"{trend}",
    #         smooth_window,
    #         "Log Return",
    #         f"Google Trend ({trend})",
    #         f"H1 {year}: {coin_name} Log Return vs Trend (rolling window={smooth_window})",
    #     )
    #     analyser.plot_smoothed(
    #         "H2",
    #         "log_return",
    #         f"{trend}",
    #         smooth_window,
    #         "Log Return",
    #         f"Google Trend ({trend})",
    #         f"H2 {year}: {coin_name} Log Return vs Trend (rolling window={smooth_window})",
    #     )
    #     analyser.plot_smoothed(
    #         "H1",
    #         "abs_log_return",
    #         f"{trend}",
    #         smooth_window,
    #         "Log Return",
    #         f"Google Trend ({trend})",
    #         f"H1 {year}: {coin_name} abs Log Return vs Trend (rolling window={smooth_window})",
    #     )
    #     analyser.plot_smoothed(
    #         "H2",
    #         "abs_log_return",
    #         f"{trend}",
    #         smooth_window,
    #         "Log Return",
    #         f"Google Trend ({trend})",
    #         f"H2 {year}: {coin_name} abs Log Return vs Trend (rolling window={smooth_window})",
    #     )
    #     analyser.plot_smoothed(
    #         "H1",
    #         "log_return",
    #         "trend_log_return",
    #         smooth_window,
    #         "Price Log Return",
    #         "Trend Log Return",
    #         f"H1 {year}: {coin_name} Price Log Return vs Trend Log Return (rolling window={smooth_window})",
    #     )
    #     analyser.plot_smoothed(
    #         "H2",
    #         "log_return",
    #         "trend_log_return",
    #         smooth_window,
    #         "Price Log Return",
    #         "Trend Log Return",
    #         f"H2 {year}: {coin_name} Price Log Return vs Trend Log Return (rolling window={smooth_window})",
    #     )
    # analyser.plot_smoothed(
    #     "H1",
    #     "abs_log_return",
    #     "abs_trend_log_return",
    #     smooth_window,
    #     "Price Abs Log Return",
    #     f"Trend Abs Log Return({trend})",
    #     f"H1 {year}: {coin_name} Price Abs Log Return vs Trend Abs Log Return (rolling window={smooth_window})",
    # )
    # analyser.plot_smoothed(
    #     "H2",
    #     "abs_log_return",
    #     "abs_trend_log_return",
    #     smooth_window,
    #     "Price Abs Log Return",
    #     f"Trend Abs Log Return ({trend})",
    #     f"H2 {year}: {coin_name} Price Abs Log Return vs Trend Abs Log Return (rolling window={smooth_window})",
    # )


=== Testing Granger causality up to lag=3 in 2021 ===

>> Does TREND Granger-cause PRICE?

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.3619  , p=0.5482  , df_denom=177, df_num=1
ssr based chi2 test:   chi2=0.3680  , p=0.5441  , df=1
likelihood ratio test: chi2=0.3677  , p=0.5443  , df=1
parameter F test:         F=0.3619  , p=0.5482  , df_denom=177, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=1.9772  , p=0.1416  , df_denom=174, df_num=2
ssr based chi2 test:   chi2=4.0679  , p=0.1308  , df=2
likelihood ratio test: chi2=4.0224  , p=0.1338  , df=2
parameter F test:         F=1.9772  , p=0.1416  , df_denom=174, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=1.4000  , p=0.2446  , df_denom=171, df_num=3
ssr based chi2 test:   chi2=4.3718  , p=0.2240  , df=3
likelihood ratio test: chi2=4.3190  , p=0.2290  , df=3
parameter F test:         F=1.4000  , p=0.2446  , df_denom=171, df_num=3

>>

/Users/dsf-air-17/Dev/nodiens_index/venv/lib/python3.13/site-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/Users/dsf-air-17/Dev/nodiens_index/venv/lib/python3.13/site-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/Users/dsf-air-17/Dev/nodiens_index/venv/lib/python3.13/site-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/Users/dsf-air-17/Dev/nodiens_index/venv/lib/python3.13/site-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/Users/dsf-air-17/Dev/nodiens_index/venv/lib/python3.13/site-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(


In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import grangercausalitytests
from nodiensenv.constants import DATA_DIR, FIGURE_DIR
from nodiensenv.analyser import TrendPriceAnalyser

years = [2021, 2022, 2023, 2024]

for year in years:
    trend = "bitcoin"
    coin_name = "BTC"
    trend_h1 = pd.read_csv(
        DATA_DIR / f"trend_{coin_name}_{year}_H1.csv", parse_dates=["date"]
    )
    trend_h2 = pd.read_csv(
        DATA_DIR / f"trend_{coin_name}_{year}_H2.csv", parse_dates=["date"]
    )
    price = pd.read_csv(DATA_DIR / f"{coin_name}_price_mcap.csv", parse_dates=["date"])

    # Filter price to {year}, split H1 & H2, and compute daily log-returns
    price_year = price[price["date"].dt.year == year].sort_values("date")
    price_h1 = price_year[(price_year["date"] <= f"{year}-06-30")].copy()
    price_h2 = price_year[(price_year["date"] >= f"{year}-07-01")].copy()

    for df in (price_h1, price_h2):
        df["log_return"] = np.log(df["close"] / df["close"].shift(1)) * 100
        df["abs_log_return"] = abs(df["log_return"])

    for df in (trend_h1, trend_h2):
        df["trend_log_return"] = np.log(df[f"{trend}"] / df[f"{trend}"].shift(1)) * 100
        df["abs_trend_log_return"] = abs(df["trend_log_return"])

    # Merge each half’s trends with its prices
    df_h1 = pd.merge(
        trend_h1,
        price_h1[["date", "close", "log_return", "abs_log_return"]],
        on="date",
        how="inner",
    )
    df_h2 = pd.merge(
        trend_h2,
        price_h2[["date", "close", "log_return", "abs_log_return"]],
        on="date",
        how="inner",
    )

    # series_price_h1 = df_h1[["log_return"]].dropna()
    # series_trend_h1 = df_h1[["trend_log_return"]].dropna()
    series_price_h1 = df_h1[["close"]].dropna()
    series_trend_h1 = df_h1[[f"{trend}"]].dropna()
    df_gc_h1 = pd.concat([series_price_h1, series_trend_h1], axis=1).dropna()
    df_gc_h1.columns = ["price_ret", "trend_ret"]

    series_price_h2 = df_h2[["close"]].dropna()
    series_trend_h2 = df_h2[[f"{trend}"]].dropna()
    df_gc_h2 = pd.concat([series_price_h2, series_trend_h2], axis=1).dropna()
    df_gc_h2.columns = ["price_ret", "trend_ret"]

    # Runs Granger tests up to `max_lag` for both directions:
    #     1) Does trend --> price?
    #     2) Does price --> trend?

    def run_granger(df, max_lag):
        print(f"\n=== Testing Granger causality up to lag={max_lag} in {year} ===\n")

        # 3a) trend --> price
        print(">> Does TREND Granger-cause PRICE?")
        test_result_1 = grangercausalitytests(
            df[["price_ret", "trend_ret"]], maxlag=max_lag, verbose=True
        )
        # 3b) price --> trend
        print("\n>> Does PRICE Granger-cause TREND?")
        test_result_2 = grangercausalitytests(
            df[["trend_ret", "price_ret"]], maxlag=max_lag, verbose=True
        )

        return test_result_1, test_result_2

    res_trend_to_price, res_price_to_trend = run_granger(df_gc_h1, max_lag=3)
    res_trend_to_price, res_price_to_trend = run_granger(df_gc_h2, max_lag=3)

    # analyser = TrendPriceAnalyser(trend, df_h1, df_h2)

    # for smooth_window in [3]:
    #         analyser.plot_smoothed(
    #             "H1",
    #             "close",
    #             f"{trend}",
    #             smooth_window,
    #             "Close Price",
    #             f"Google Trend ({trend})",
    #             f"H1 {year}: {coin_name} Close vs Trend (rolling window={smooth_window})",
    #         )
    #         analyser.plot_smoothed(
    #             "H2",
    #             "close",
    #             f"{trend}",
    #             smooth_window,
    #             "Close Price",
    #             f"Google Trend ({trend})",
    #             f"H2 {year}: {coin_name} Close vs Trend (rolling window={smooth_window})",
    #         )
    #     analyser.plot_smoothed(
    #         "H1",
    #         "log_return",
    #         f"{trend}",
    #         smooth_window,
    #         "Log Return",
    #         f"Google Trend ({trend})",
    #         f"H1 {year}: {coin_name} Log Return vs Trend (rolling window={smooth_window})",
    #     )
    #     analyser.plot_smoothed(
    #         "H2",
    #         "log_return",
    #         f"{trend}",
    #         smooth_window,
    #         "Log Return",
    #         f"Google Trend ({trend})",
    #         f"H2 {year}: {coin_name} Log Return vs Trend (rolling window={smooth_window})",
    #     )
    #     analyser.plot_smoothed(
    #         "H1",
    #         "abs_log_return",
    #         f"{trend}",
    #         smooth_window,
    #         "Log Return",
    #         f"Google Trend ({trend})",
    #         f"H1 {year}: {coin_name} abs Log Return vs Trend (rolling window={smooth_window})",
    #     )
    #     analyser.plot_smoothed(
    #         "H2",
    #         "abs_log_return",
    #         f"{trend}",
    #         smooth_window,
    #         "Log Return",
    #         f"Google Trend ({trend})",
    #         f"H2 {year}: {coin_name} abs Log Return vs Trend (rolling window={smooth_window})",
    #     )
    #     analyser.plot_smoothed(
    #         "H1",
    #         "log_return",
    #         "trend_log_return",
    #         smooth_window,
    #         "Price Log Return",
    #         "Trend Log Return",
    #         f"H1 {year}: {coin_name} Price Log Return vs Trend Log Return (rolling window={smooth_window})",
    #     )
    #     analyser.plot_smoothed(
    #         "H2",
    #         "log_return",
    #         "trend_log_return",
    #         smooth_window,
    #         "Price Log Return",
    #         "Trend Log Return",
    #         f"H2 {year}: {coin_name} Price Log Return vs Trend Log Return (rolling window={smooth_window})",
    #     )
    # analyser.plot_smoothed(
    #     "H1",
    #     "abs_log_return",
    #     "abs_trend_log_return",
    #     smooth_window,
    #     "Price Abs Log Return",
    #     f"Trend Abs Log Return({trend})",
    #     f"H1 {year}: {coin_name} Price Abs Log Return vs Trend Abs Log Return (rolling window={smooth_window})",
    # )
    # analyser.plot_smoothed(
    #     "H2",
    #     "abs_log_return",
    #     "abs_trend_log_return",
    #     smooth_window,
    #     "Price Abs Log Return",
    #     f"Trend Abs Log Return ({trend})",
    #     f"H2 {year}: {coin_name} Price Abs Log Return vs Trend Abs Log Return (rolling window={smooth_window})",
    # )


=== Testing Granger causality up to lag=3 in 2021 ===

>> Does TREND Granger-cause PRICE?

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.3619  , p=0.5482  , df_denom=177, df_num=1
ssr based chi2 test:   chi2=0.3680  , p=0.5441  , df=1
likelihood ratio test: chi2=0.3677  , p=0.5443  , df=1
parameter F test:         F=0.3619  , p=0.5482  , df_denom=177, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=1.9772  , p=0.1416  , df_denom=174, df_num=2
ssr based chi2 test:   chi2=4.0679  , p=0.1308  , df=2
likelihood ratio test: chi2=4.0224  , p=0.1338  , df=2
parameter F test:         F=1.9772  , p=0.1416  , df_denom=174, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=1.4000  , p=0.2446  , df_denom=171, df_num=3
ssr based chi2 test:   chi2=4.3718  , p=0.2240  , df=3
likelihood ratio test: chi2=4.3190  , p=0.2290  , df=3
parameter F test:         F=1.4000  , p=0.2446  , df_denom=171, df_num=3

>>

/Users/dsf-air-17/Dev/nodiens_index/venv/lib/python3.13/site-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/Users/dsf-air-17/Dev/nodiens_index/venv/lib/python3.13/site-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/Users/dsf-air-17/Dev/nodiens_index/venv/lib/python3.13/site-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/Users/dsf-air-17/Dev/nodiens_index/venv/lib/python3.13/site-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/Users/dsf-air-17/Dev/nodiens_index/venv/lib/python3.13/site-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(


In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import grangercausalitytests
from nodiensenv.constants import DATA_DIR, FIGURE_DIR
from nodiensenv.analyser import TrendPriceAnalyser

years = [2021, 2022, 2023, 2024]

for year in years:
    trend = "bitcoin"
    coin_name = "BTC"
    trend_h1 = pd.read_csv(
        DATA_DIR / f"trend_{coin_name}_{year}_H1.csv", parse_dates=["date"]
    )
    trend_h2 = pd.read_csv(
        DATA_DIR / f"trend_{coin_name}_{year}_H2.csv", parse_dates=["date"]
    )
    price = pd.read_csv(DATA_DIR / f"{coin_name}_price_mcap.csv", parse_dates=["date"])

    # Filter price to {year}, split H1 & H2, and compute daily log-returns
    price_year = price[price["date"].dt.year == year].sort_values("date")
    price_h1 = price_year[(price_year["date"] <= f"{year}-06-30")].copy()
    price_h2 = price_year[(price_year["date"] >= f"{year}-07-01")].copy()

    for df in (price_h1, price_h2):
        df["log_return"] = np.log(df["close"] / df["close"].shift(1)) * 100
        df["abs_log_return"] = abs(df["log_return"])

    for df in (trend_h1, trend_h2):
        df["trend_log_return"] = np.log(df[f"{trend}"] / df[f"{trend}"].shift(1)) * 100
        df["abs_trend_log_return"] = abs(df["trend_log_return"])

    # Merge each half’s trends with its prices
    df_h1 = pd.merge(
        trend_h1,
        price_h1[["date", "close", "log_return", "abs_log_return"]],
        on="date",
        how="inner",
    )
    df_h2 = pd.merge(
        trend_h2,
        price_h2[["date", "close", "log_return", "abs_log_return"]],
        on="date",
        how="inner",
    )

    def run_granger(df, max_lag):
        print(f"\n=== Testing Granger causality up to lag={max_lag} in {year} ===\n")

        # 3a) trend --> price
        print(">> Does TREND Granger-cause PRICE?")
        test_result_1 = grangercausalitytests(
            df[["price_ret", "trend_ret"]], maxlag=max_lag, verbose=True
        )
        # 3b) price --> trend
        print("\n>> Does PRICE Granger-cause TREND?")
        test_result_2 = grangercausalitytests(
            df[["trend_ret", "price_ret"]], maxlag=max_lag, verbose=True
        )

        return test_result_1, test_result_2

    # series_price_h1 = df_h1[["log_return"]].dropna()
    # series_trend_h1 = df_h1[["trend_log_return"]].dropna()

    for h in ("H1", "H2"):
        df = df_h1 if h == "H1" else df_h2
        series_price = df[["close"]].dropna()
        series_trend = df[[f"{trend}"]].dropna()
        df_gc = pd.concat([series_price, series_trend], axis=1).dropna()
        df_gc.columns = ["price_ret", "trend_ret"]
        print(f"\nGranger causality results for {h} in {year}:")
        res_trend_to_price, res_price_to_trend = run_granger(df_gc, max_lag=3)


    # series_price_h1 = df_h1[["close"]].dropna()
    # series_trend_h1 = df_h1[[f"{trend}"]].dropna()
    # df_gc_h1 = pd.concat([series_price_h1, series_trend_h1], axis=1).dropna()
    # df_gc_h1.columns = ["price_ret", "trend_ret"]

    # series_price_h2 = df_h2[["close"]].dropna()
    # series_trend_h2 = df_h2[[f"{trend}"]].dropna()
    # df_gc_h2 = pd.concat([series_price_h2, series_trend_h2], axis=1).dropna()
    # df_gc_h2.columns = ["price_ret", "trend_ret"]

    # # Runs Granger tests up to `max_lag` for both directions:
    # #     1) Does trend --> price?
    # #     2) Does price --> trend?

    # def run_granger(df, max_lag):
    #     print(f"\n=== Testing Granger causality up to lag={max_lag} in {year} ===\n")

    #     # 3a) trend --> price
    #     print(">> Does TREND Granger-cause PRICE?")
    #     test_result_1 = grangercausalitytests(
    #         df[["price_ret", "trend_ret"]], maxlag=max_lag, verbose=True
    #     )
    #     # 3b) price --> trend
    #     print("\n>> Does PRICE Granger-cause TREND?")
    #     test_result_2 = grangercausalitytests(
    #         df[["trend_ret", "price_ret"]], maxlag=max_lag, verbose=True
    #     )

    #     return test_result_1, test_result_2

    # res_trend_to_price, res_price_to_trend = run_granger(df_gc_h1, max_lag=3)
    # res_trend_to_price, res_price_to_trend = run_granger(df_gc_h2, max_lag=3)

    # analyser = TrendPriceAnalyser(trend, df_h1, df_h2)

    # for smooth_window in [3]:
    #         analyser.plot_smoothed(
    #             "H1",
    #             "close",
    #             f"{trend}",
    #             smooth_window,
    #             "Close Price",
    #             f"Google Trend ({trend})",
    #             f"H1 {year}: {coin_name} Close vs Trend (rolling window={smooth_window})",
    #         )
    #         analyser.plot_smoothed(
    #             "H2",
    #             "close",
    #             f"{trend}",
    #             smooth_window,
    #             "Close Price",
    #             f"Google Trend ({trend})",
    #             f"H2 {year}: {coin_name} Close vs Trend (rolling window={smooth_window})",
    #         )
    #     analyser.plot_smoothed(
    #         "H1",
    #         "log_return",
    #         f"{trend}",
    #         smooth_window,
    #         "Log Return",
    #         f"Google Trend ({trend})",
    #         f"H1 {year}: {coin_name} Log Return vs Trend (rolling window={smooth_window})",
    #     )
    #     analyser.plot_smoothed(
    #         "H2",
    #         "log_return",
    #         f"{trend}",
    #         smooth_window,
    #         "Log Return",
    #         f"Google Trend ({trend})",
    #         f"H2 {year}: {coin_name} Log Return vs Trend (rolling window={smooth_window})",
    #     )
    #     analyser.plot_smoothed(
    #         "H1",
    #         "abs_log_return",
    #         f"{trend}",
    #         smooth_window,
    #         "Log Return",
    #         f"Google Trend ({trend})",
    #         f"H1 {year}: {coin_name} abs Log Return vs Trend (rolling window={smooth_window})",
    #     )
    #     analyser.plot_smoothed(
    #         "H2",
    #         "abs_log_return",
    #         f"{trend}",
    #         smooth_window,
    #         "Log Return",
    #         f"Google Trend ({trend})",
    #         f"H2 {year}: {coin_name} abs Log Return vs Trend (rolling window={smooth_window})",
    #     )
    #     analyser.plot_smoothed(
    #         "H1",
    #         "log_return",
    #         "trend_log_return",
    #         smooth_window,
    #         "Price Log Return",
    #         "Trend Log Return",
    #         f"H1 {year}: {coin_name} Price Log Return vs Trend Log Return (rolling window={smooth_window})",
    #     )
    #     analyser.plot_smoothed(
    #         "H2",
    #         "log_return",
    #         "trend_log_return",
    #         smooth_window,
    #         "Price Log Return",
    #         "Trend Log Return",
    #         f"H2 {year}: {coin_name} Price Log Return vs Trend Log Return (rolling window={smooth_window})",
    #     )
    # analyser.plot_smoothed(
    #     "H1",
    #     "abs_log_return",
    #     "abs_trend_log_return",
    #     smooth_window,
    #     "Price Abs Log Return",
    #     f"Trend Abs Log Return({trend})",
    #     f"H1 {year}: {coin_name} Price Abs Log Return vs Trend Abs Log Return (rolling window={smooth_window})",
    # )
    # analyser.plot_smoothed(
    #     "H2",
    #     "abs_log_return",
    #     "abs_trend_log_return",
    #     smooth_window,
    #     "Price Abs Log Return",
    #     f"Trend Abs Log Return ({trend})",
    #     f"H2 {year}: {coin_name} Price Abs Log Return vs Trend Abs Log Return (rolling window={smooth_window})",
    # )


Granger causality results for H1 in 2021:

=== Testing Granger causality up to lag=3 in 2021 ===

>> Does TREND Granger-cause PRICE?

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.3619  , p=0.5482  , df_denom=177, df_num=1
ssr based chi2 test:   chi2=0.3680  , p=0.5441  , df=1
likelihood ratio test: chi2=0.3677  , p=0.5443  , df=1
parameter F test:         F=0.3619  , p=0.5482  , df_denom=177, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=1.9772  , p=0.1416  , df_denom=174, df_num=2
ssr based chi2 test:   chi2=4.0679  , p=0.1308  , df=2
likelihood ratio test: chi2=4.0224  , p=0.1338  , df=2
parameter F test:         F=1.9772  , p=0.1416  , df_denom=174, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=1.4000  , p=0.2446  , df_denom=171, df_num=3
ssr based chi2 test:   chi2=4.3718  , p=0.2240  , df=3
likelihood ratio test: chi2=4.3190  , p=0.2290  , df=3
parameter F test:         F=1.400

/Users/dsf-air-17/Dev/nodiens_index/venv/lib/python3.13/site-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/Users/dsf-air-17/Dev/nodiens_index/venv/lib/python3.13/site-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/Users/dsf-air-17/Dev/nodiens_index/venv/lib/python3.13/site-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/Users/dsf-air-17/Dev/nodiens_index/venv/lib/python3.13/site-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/Users/dsf-air-17/Dev/nodiens_index/venv/lib/python3.13/site-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
